In [10]:
import dill

In [11]:
import importlib.util
import sys
import os

module_path = os.path.abspath("../1_datasets/USAVars.py")

spec = importlib.util.spec_from_file_location("USAVars", module_path)
module = importlib.util.module_from_spec(spec)
sys.modules["USAVars"] = module
spec.loader.exec_module(module)

USAVars = module.USAVars


In [12]:
train_data = USAVars(root="/share/usavars", isTrain=True)

In [21]:
cost_path = f"/home/libe2152/optimizedsampling/0_data/costs/usavars/{label}/convenience_costs/linear_distance_based_costs_top20_urban_0.001.pkl"

In [22]:
with open(cost_path, "rb") as f:
    arrs = dill.load(f)

In [23]:
ids = arrs['ids']
costs = arrs['costs']

In [24]:
ids_to_costs = dict(zip(ids, costs))

In [25]:
ids_to_costs

{'1535,2258': np.float64(339.8870853282667),
 '900,3142': np.float64(175.0164861514327),
 '1851,2819': np.float64(375.98926808129255),
 '1692,3161': np.float64(295.7166972300075),
 '1119,2918': np.float64(403.0715611512146),
 '1020,2271': np.float64(525.5802512536366),
 '620,1534': np.float64(506.28530009251494),
 '858,2628': np.float64(76.64816341616712),
 '1082,2833': np.float64(308.84425677249965),
 '541,767': np.float64(663.1287717445131),
 '1047,521': np.float64(451.80063396751075),
 '126,1870': np.float64(628.8372003229919),
 '1707,2425': np.float64(472.8405722282782),
 '1609,1974': np.float64(123.01610451216342),
 '1683,1699': np.float64(434.78283923062617),
 '1637,2121': np.float64(124.00245464362553),
 '619,2894': np.float64(248.72163677753298),
 '679,2552': np.float64(256.6111971367078),
 '1755,1101': np.float64(262.8418294174673),
 '1467,3140': np.float64(286.2645886960827),
 '205,2138': np.float64(395.962684263178),
 '2104,1979': np.float64(319.48973689688285),
 '1582,2346'

In [26]:
costs = [ids_to_costs[id_] for id_ in train_data.ids]

In [27]:
import geopandas as gpd

In [ ]:
def plot(
    costs,
    country_shape_file: str = '/home/libe2152/optimizedsampling/0_data/boundaries/us/us_states_provinces/ne_110m_admin_1_states_provinces.shp',
    exclude_names: list[str] = ['Alaska', 'Hawaii', 'Puerto Rico'],
    point_color: str = 'red',
    point_size: float = 3,
):
    """
    Plot selected lat/lon points on a country shapefile.

    Args:
        indices: list of indices in self.latlons to plot.
        country_shape_file: path to a shapefile for plotting the country boundary.
        country_name: optional name to filter a specific country (must match shapefile's attribute).
        exclude_names: optional list of names to exclude (e.g., overseas territories).
        point_color: color of plotted points.
        point_size: size of plotted points.
        title: optional title for the plot.

    Returns:
        A matplotlib Figure showing the points on the map.
    """
    print("Plotting latlon subset...")
    # Load the country shapefile
    country = gpd.read_file(country_shape_file)

    if country_name is not None and 'NAME' in country.columns:
        country = country[country['NAME'] == country_name]

    if exclude_names:
        country = country[~country['name'].isin(exclude_names)]

    # Create GeoDataFrame of selected lat/lons
    latlons_subset = train_data.latlons
    points_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(latlons_subset[:, 1], latlons_subset[:, 0]), crs='EPSG:4326')

    # Plot
    fig, ax = plt.subplots(figsize=(12, 10))
    country.plot(ax=ax, edgecolor='black', facecolor='none')
    points_gdf.plot(ax=ax, color=point_color, markersize=point_size)

    ax.set_axis_off()
    plt.show()

In [ ]:
plot